In [1]:
import pandas as pd
import numpy as np
import torch

In [10]:
label = pd.read_pickle("urfunny/humor_label_sdk.pkl")
label


{1: 1,
 3: 1,
 4: 1,
 5: 1,
 7: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 0,
 13: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 22: 0,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 0,
 33: 0,
 35: 0,
 36: 0,
 38: 0,
 39: 0,
 40: 1,
 41: 0,
 43: 0,
 45: 1,
 46: 1,
 48: 0,
 49: 0,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 59: 1,
 61: 1,
 62: 1,
 63: 1,
 66: 0,
 68: 0,
 69: 0,
 70: 1,
 71: 1,
 72: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 82: 1,
 83: 1,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 94: 0,
 95: 0,
 96: 0,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 103: 1,
 104: 1,
 108: 0,
 109: 0,
 110: 0,
 112: 0,
 113: 0,
 114: 0,
 117: 0,
 118: 1,
 120: 1,
 121: 0,
 122: 0,
 123: 0,
 125: 1,
 127: 1,
 128: 1,
 129: 1,
 131: 0,
 133: 0,
 134: 0,
 135: 0,
 136: 1,
 137: 1,
 139: 1,
 140: 1,
 142: 1,
 144: 0,
 145: 0,
 146: 0,
 148: 0,
 151: 1,
 152: 1,
 153: 0,
 154: 0,
 156: 1,
 158: 1,
 159: 1,
 160: 1,
 162: 1,
 163: 1,
 165: 1,
 166: 1,
 

In [2]:
class Trainer:
    """ big_batch: if batch > big_batch then we will use checkpointing to save memory.
        num_steps: number of steps to divide an epoch into. Used in conjunction with patience for early stopping
    """
    def __init__(self , big_batch: int , num_steps : int , save : bool =  False) -> None:
        
        self.big_batch = big_batch
        self.num_steps = num_steps
        self.save = save
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.patient_iter = 0    

In [23]:
full_df = pd.read_pickle("urfunny/language_sdk.pkl")
full_df = pd.DataFrame(full_df).T.reset_index().rename(columns = {"index" : "file_id"})


In [24]:
full_df.drop(columns=["punchline_embedding_indexes" , "context_embedding_indexes"] , inplace=True)
full_df

,file_id,punchline_sentence,context_sentences,punchline_features,context_features,punchline_intervals,context_intervals,root,label
0,1,the cat likes this version better,[the mathematics of quantum mechanics very acc...,"[the, cat, likes, this, version, better]","[[the, mathematics, of, quantum, mechanics, ve...","[[19.985, 20.044], [20.144, 20.344], [20.344, ...","[[[0.0, 0.07], [0.07, 0.509], [0.509, 0.589], ...",371,1
1,3,a lot of people in this room make a good livin...,[but physics does tell us what particles can b...,"[a, lot, of, people, in, this, room, make, a, ...","[[but, physics, does, tell, us, what, particle...","[[35.779, 35.809], [35.809, 35.938], [35.938, ...","[[[0.0, 0.11], [0.11, 0.449], [0.449, 0.659], ...",371,1
2,4,there is also a very weak force called rather ...,[electron also has a neutral partner called th...,"[there, is, also, a, very, weak, force, called...","[[electron, also, has, a, neutral, partner, ca...","[[35.868, 35.988], [35.988, 36.078], [36.078, ...","[[[0.0, 0.289], [0.289, 0.499], [0.499, 0.589]...",371,1
3,5,and of course bad for me personally,"[circles and other geometrical figures, of cou...","[and, of, course, bad, for, me, personally]","[[circles, and, other, geometrical, figures], ...","[[57.429, 57.829], [57.829, 57.958], [57.958, ...","[[[0.0, 0.479], [0.539, 0.678], [0.678, 0.868]...",371,1
4,7,i've tried the best i can to live a balanced life,[predicting how nature works is a very risky g...,"[i've, tried, the, best, i, can, to, live, a, ...","[[predicting, how, nature, works, is, a, very,...","[[38.343, 38.453], [38.453, 38.662], [38.662, ...","[[[0.0, 0.269], [0.269, 0.349], [0.349, 0.469]...",371,1
...,...,...,...,...,...,...,...,...,...
10161,15613,one of the things i want to do is to help you ...,[you're not interested in the limits of these ...,"[one, of, the, things, i, want, to, do, is, to...","[[you're, not, interested, in, the, limits, of...","[[16.183, 16.333], [16.333, 16.433], [16.433, ...","[[[0.0, 0.11], [0.11, 0.319], [0.319, 0.818], ...",845,1
10162,15615,i want you to read this because we're going to...,[that is you know what simplicity is you know ...,"[i, want, you, to, read, this, because, we're,...","[[that, is, you, know, what, simplicity, is, y...","[[12.212, 12.252], [12.252, 12.432], [12.432, ...","[[[0.0, 0.16], [0.16, 0.469], [0.469, 0.599], ...",845,0
10163,15616,a coffee cup we don't think about coffee cups ...,[i want you to read this because we're going t...,"[a, coffee, cup, we, don't, think, about, coff...","[[i, want, you, to, read, this, because, we're...","[[35.459, 35.539], [35.539, 35.918], [35.918, ...","[[[0.0, 0.06], [0.06, 0.239], [0.239, 0.339], ...",845,0
10164,15618,and my body had clearly changed and matt who w...,[i want to start with a story a la seth godin ...,"[and, my, body, had, clearly, changed, and, ma...","[[i, want, to, start, with, a, story, a, la, s...","[[26.949, 27.657], [27.827, 27.936], [27.936, ...","[[[0.0, 0.11], [0.11, 0.259], [0.259, 0.319], ...",91,1


In [31]:
full_df['punchline_intervals'].iloc[0]

[[19.985, 20.044],
 [20.144, 20.344],
 [20.344, 20.533],
 [20.533, 20.673],
 [20.673, 20.952],
 [20.952, 21.232]]

In [32]:
full_df['punchline_timings'] = full_df['punchline_intervals'].apply(lambda x:  x[-1][1] - x[0][0] )
full_df['context_timings'] = full_df['context_intervals'].apply(lambda x: x[-1][1] - x[0][0] )

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [ ]:
full_df['punchline_timings'].value_counts()

In [4]:
df = pd.read_pickle("urfunny/data_folds.pkl")
df

{'train': [1,
  3,
  4,
  5,
  7,
  9,
  10,
  11,
  12,
  13,
  15,
  16,
  17,
  18,
  19,
  20,
  22,
  40,
  41,
  43,
  45,
  46,
  50,
  52,
  53,
  54,
  55,
  56,
  57,
  59,
  61,
  62,
  63,
  66,
  68,
  69,
  70,
  71,
  72,
  75,
  76,
  77,
  78,
  82,
  83,
  84,
  85,
  86,
  87,
  89,
  90,
  91,
  92,
  94,
  95,
  96,
  118,
  120,
  121,
  122,
  123,
  125,
  127,
  128,
  129,
  131,
  133,
  134,
  135,
  136,
  137,
  139,
  140,
  142,
  144,
  145,
  146,
  148,
  151,
  152,
  153,
  154,
  156,
  158,
  159,
  160,
  162,
  163,
  165,
  166,
  167,
  168,
  170,
  171,
  173,
  174,
  176,
  177,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  204,
  205,
  207,
  208,
  209,
  210,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  220,
  221,
  223,
  224,
  225,
  227,
  228,
  229,
  232,
  235,
  237,
  239,
  240,
  242,
  245,
  248,
  250,
  252,
  253,
  254,
  255,
  256,
  259,
  260,
  262,
  263,
  265,
  269,
  279,
  280,
  281,
  282,
  2

In [8]:
df.keys()

dict_keys(['train', 'dev', 'test'])

In [9]:
len(df['train']) + len(df['test']) + len(df['dev'])

9588

In [5]:
# Reset the index
df = df.reset_index()

# Use melt to unpivot the DataFrame
df_melted = df.melt(id_vars='index', var_name='split')
df_melted

AttributeError: 'dict' object has no attribute 'reset_index'